In [1]:
sourcepath = r'D:\jupyterWorkspace\data\dataset\VulDeePecker-master/'
import os
os.listdir(sourcepath)
sourcepath

import json
setting = json.load(open("config.json"))

setting
list(setting["dataset"].keys())

['cwe-119', 'cwe-399']

In [1]:
import argparse
import json
from typing import List

import torch
import torch.utils.data as data
from blstm import BLSTM
from cache import load_cache, store_cache
from cgd import CGDDataset
from config import DefaultTrainConfig
from fit import Fitter

setting = json.load(open("config.json"))

def load_dataset(category: str) -> data.Dataset:
    cache_name = f"{category}_dataset"
    cgd_dataset = load_cache(cache_name)
    if cgd_dataset is None:
        cgd_dataset = CGDDataset(
            setting["dataset"][category]["cgd_filepath"],
            setting["hyperparameter"]["input_size"],
        )
        store_cache(cache_name, ["cgd.py", "config.json"], cgd_dataset)

    return cgd_dataset


def calc_weight(dataset):
    total_step = len(dataset)
    weight = [0, 0]
    for _, label in dataset:
        weight[label] += 1

    weight[0], weight[1] = weight[1], weight[0]
    return torch.tensor(list(map(lambda v: v / total_step, weight)))


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description="VulDeePecker implemented in pytorch. "
    )
    parser.add_argument("-cmd", choices=["train"], default='train')
    parser.add_argument("-category", choices=list(setting["dataset"].keys()), default='cwe-119')
    result = parser.parse_args()

    cmd = result.cmd
    category = result.category

    config = DefaultTrainConfig()

    if cmd == "train":
        print("[*] loading dataset...")
        dataset = load_dataset(category)
        print("[+] loading dataset complete")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = BLSTM(
            config.input_size,
            config.hidden_size,
            config.num_layers,
            config.num_classes,
            config.dropout,
            device,
        ).to(device)
        print("[*] training model...")
        fitter = Fitter(model, device, config)
        total_result = fitter.cross_validation(dataset)

        with open("./cross_val.csv", "w") as f:
            f.write(
                "fold,epoch,train_f1,val_f1,train_acc,val_acc,train_recall,val_recall,train_loss,val_loss\n"
            )
            for fold, fold_result in enumerate(total_result):
                for epoch, (
                    train_summary_loss,
                    train_total_score,
                    val_summary_loss,
                    val_total_score,
                ) in enumerate(fold_result):
                    print(
                        fold + 1,
                        epoch + 1,
                        train_total_score.f1,
                        val_total_score.f1,
                        train_total_score.precision,
                        val_total_score.precision,
                        train_total_score.recall,
                        val_total_score.recall,
                        train_summary_loss.avg,
                        val_summary_loss.avg,
                        sep=",",
                        file=f,
                    )

        print("[*] training model complete...")
        print(result)


usage: ipykernel_launcher.py [-h] [-cmd {train}] [-category {cwe-119,cwe-399}]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\ago\AppData\Roaming\jupyter\runtime\kernel-31e1d20b-f545-46ce-814f-5566bf833ab6.json


SystemExit: 2

C:\allsoftware\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 1

In [7]:

import argparse
import json
from typing import List

import torch
import torch.utils.data as data
from blstm import BLSTM
from cache import load_cache, store_cache
from cgd import CGDDataset
from config import DefaultTrainConfig
from fit import Fitter

setting = json.load(open("config.json"))


def load_dataset(category: str) -> data.Dataset:
    cache_name = f"{category}_dataset"
    cgd_dataset = load_cache(cache_name)
    if cgd_dataset is None:
        cgd_dataset = CGDDataset(
            setting["dataset"][category]["cgd_filepath"],
            setting["hyperparameter"]["input_size"],
        )
        store_cache(cache_name, ["cgd.py", "config.json"], cgd_dataset)

    return cgd_dataset


def calc_weight(dataset):
    total_step = len(dataset)
    weight = [0, 0]
    for _, label in dataset:
        weight[label] += 1

    weight[0], weight[1] = weight[1], weight[0]
    return torch.tensor(list(map(lambda v: v / total_step, weight)))


# if __name__ == "__main__":
import argparse

config = DefaultTrainConfig()

parser = argparse.ArgumentParser(
    description="VulDeePecker implemented in pytorch. "
)
subparser = parser.add_subparsers(
    help="cmd", dest="cmd", description="Command to run. "
)
subparser.required = True

train_group = subparser.add_parser("cv", help="Run cross validation. ")
train_group.add_argument("-category", choices=list(setting["dataset"].keys()), default='cwe-119')
train_group.add_argument("-l", "--num-layers", nargs="?", type=int)
train_group.add_argument("-d", "--dropout", nargs="?", type=float)
train_group.add_argument("-b", "--batch-size", nargs="?", type=int)
train_group.add_argument("-e", "--num-epochs", nargs="?", type=int)
train_group.add_argument("-lr", "--learning-rate", nargs="?", type=float)
train_group.add_argument("-k", "--k-fold", nargs="?", type=int)
train_group.add_argument("-t", "--test-size", nargs="?", type=float)
train_group.add_argument("-o", "--output", nargs="?", type=argparse.FileType("w"))

result = parser.parse_args(args=['cv'])
cmd = result.cmd
category = result.category

# if cmd == "cv":
parameters = [
    "num_layers",
    "dropout",
    "batch_size",
    "num_epochs",
    "learning_rate",
    "k_fold",
    "test_size",
]
for parameter in parameters:
    val = getattr(result, parameter)
    if val is not None:
        setattr(config, parameter, val)
#         print(parameters)
print("[*] loading dataset...")
print(category)

dataset = load_dataset(category)
print("[+] loading dataset complete")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BLSTM(
    config.input_size,
    config.hidden_size,
    config.num_layers,
    config.num_classes,
    config.dropout,
    device,
).to(device)
print("[*] training model...")
fitter = Fitter(model, device, config)
total_result = fitter.cross_validation(dataset)

[*] loading dataset...
cwe-119
[+] loading dataset complete
[*] training model...
Train Step 500/504, summary_loss: 0.68500, summary_precision: 0.76217, summary_recall: 0.45079, summary_f1: 0.56651, time: 481.79205
Epoch 1, Train F1: 0.56798, Validation F1: 0.85012, Train Acc: 0.76163, Validation Acc: 0.75958, Train Recall: 0.45285, Validation Recall: 0.96518, Train Loss: 0.68496, Validation Loss: 0.67290
Train Step 500/504, summary_loss: 0.66591, summary_precision: 0.79800, summary_recall: 0.48013, summary_f1: 0.59954, time: 468.50152
Epoch 2, Train F1: 0.59938, Validation F1: 0.65268, Train Acc: 0.79829, Validation Acc: 0.88153, Train Recall: 0.47983, Validation Recall: 0.51816, Train Loss: 0.66565, Validation Loss: 0.62906
Train Step 500/504, summary_loss: 0.62216, summary_precision: 0.89127, summary_recall: 0.49222, summary_f1: 0.63419, time: 474.74080
Epoch 3, Train F1: 0.63379, Validation F1: 0.58316, Train Acc: 0.89133, Validation Acc: 0.94005, Train Recall: 0.49172, Validation 

KeyboardInterrupt: 

In [ ]:
if result.output is None:
    f = open("./cross_val.csv", "w")
else:
    f = result.output
f.write(
    "fold,epoch,train_f1,val_f1,train_acc,val_acc,train_recall,val_recall,train_loss,val_loss\n"
)
for fold, fold_result in enumerate(total_result):
    for epoch, (
        train_summary_loss,
        train_total_score,
        val_summary_loss,
        val_total_score,
    ) in enumerate(fold_result):
        print(
            fold + 1,
            epoch + 1,
            train_total_score.f1,
            val_total_score.f1,
            train_total_score.precision,
            val_total_score.precision,
            train_total_score.recall,
            val_total_score.recall,
            train_summary_loss.avg,
            val_summary_loss.avg,
            sep=",",
            file=f,
        )
f.close()

print("[*] training model complete...")


# end

In [ ]:
# dataset = load_dataset(category)
dataset